In [34]:
import os
import sys

os.chdir('/scratch/edk202/word2gm-fast/notebooks')
sys.path.append('../src/word2gm_fast')
sys.path.append('../')

import dataprep.load_and_filter_corpus
import tests.test_load_and_filter_corpus

In [35]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestLoadAndFilterCorpus)
unittest.TextTestRunner().run(suite)

2025-06-18 21:19:07.658233: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-06-18 21:19:09.183944: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
F
FAIL: test_load_and_filter (__main__.TestLoadAndFilterCorpus.test_load_and_filter)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/scratch/edk202/word2gm-fast/tests/test_load_and_filter_corpus.py", line 31, in test_load_and_filter
    self.assertCountEqual(result_lines, self.valid_lines)
AssertionError: Element counts were not equal:
First has 1, Second has 0:  'UNK quick brown fox jumps'

----------------------------------------------------------------------
Ran 1 test in 1.548s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>